In [1]:
import numpy as np

In [2]:
import logging
logging.basicConfig(level=logging.INFO, format='[%(asctime)s] %(levelname)s: %(message)s')

In [3]:
!wget https://raw.githubusercontent.com/raeubaen/RadioMonteCarlo/main/tf_keras_model.py

--2022-06-15 13:28:08--  https://raw.githubusercontent.com/raeubaen/RadioMonteCarlo/main/tf_keras_model.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7649 (7.5K) [text/plain]
Saving to: ‘tf_keras_model.py’

tf_keras_model.py   100%[===================>]   7.47K  --.-KB/s    in 0s      

2022-06-15 13:28:08 (36.7 MB/s) - ‘tf_keras_model.py’ saved [7649/7649]



In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# VANNO TUTTI CAMBIATI

# !wget https://www.dropbox.com/s/6s5khncrval4rz9/bkg_et.npy?dl=1 -o bkg_et.npy
# !wget https://www.dropbox.com/s/qc7z1nhqh5xvpbg/bkg_mask.npy?dl=1 -o bkg_mask.npy
# !wget https://www.dropbox.com/s/fs5e9wuhqtr66xh/bkg_xy.npy?dl=1 -o bkg_xy.npy
# !wget https://www.dropbox.com/s/w0b94xzyk2wg7y4/signal_et.npy?dl=1 -o signal_et.npy
# !wget https://www.dropbox.com/s/3k6b0u3zz65xnyf/signal_mask.npy?dl=1 -o signal_mask.npy
# !wget https://www.dropbox.com/s/chcep3q8terrj4k/signal_xy.npy?dl=1 -o signal_xy.npy

In [5]:
import tensorflow as tf
from tensorflow import keras
from tf_keras_model import get_particle_net, get_particle_net_lite, edge_conv

[2022-06-15 13:28:36,559] INFO: NumExpr defaulting to 2 threads.


In [6]:
data_folder = "/content/drive/MyDrive/datiML"

name_map = {"points": "rphi", "features": "et", "mask": "mask"} # va provato sia xy che rphi


signal  = {key: np.load(f"{data_folder}/signal_{ name_map[key]}.npy") for key in name_map}

mnbs    = {key: np.load(f"{data_folder}/mnbs_{   name_map[key]}.npy") for key in name_map}

cosmics = {key: np.load(f"{data_folder}/cosmics_{name_map[key]}.npy") for key in name_map}

In [8]:
S =  signal["mask"].shape[0]
F =    mnbs["mask"].shape[0]
C = cosmics["mask"].shape[0]

N = S + F + C

p = np.random.permutation(N)

NameError: ignored

In [ ]:
'''
cluster su disco 0 sopra 50 MeV in un injection cycle
RMC: 66
MNBS: 107k
cosmici: 4

cluster simulati su disco 0 sopra i 50 MeV:
RMC: 150k
MNBS: 16505 * 5 = 82k
cosmici: 53k

pesi:
RMC: 66/150k 
MNBS: 107k/82k
Cosmici: 4/53k

In [ ]:
data = {key: np.concatenate( (signal[key], bkg[key]) )[p] for key in name_map}

label = np.concatenate( ( np.ones( (S) ), np.zeros( (B) ) ) )[p]

In [ ]:
train_data  = {key: data[key][int(N*0.0) : int(N*0.7)] for key in name_map}
test_data   = {key: data[key][int(N*0.7) : int(N*0.9)] for key in name_map}
val_data    = {key: data[key][int(N*0.9) : int(N*1.0)] for key in name_map}

train_label = label[int(N*0.0) : int(N*0.7)]
test_label  = label[int(N*0.7) : int(N*0.9)]
val_label   = label[int(N*0.9) : int(N*1.0)]

In [ ]:
model_type = 'particle_net_lite' # choose between 'particle_net' and 'particle_net_lite'
num_classes = 2
input_shapes = {'points': (50, 2), 'features': (50, 2), 'mask':(50, 1)}

if 'lite' in model_type:
    model = get_particle_net_lite(num_classes, input_shapes)
else:
    model = get_particle_net(num_classes, input_shapes)

In [ ]:
# Training parameters
batch_size = 1024 if 'lite' in model_type else 384
epochs = 30

In [ ]:
def lr_schedule(epoch):
    lr = 1e-3
    if epoch > 10:
        lr *= 0.1
    elif epoch > 20:
        lr *= 0.01
    logging.info('Learning rate: %f'%lr)
    return lr

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer=keras.optimizers.Adam(learning_rate=lr_schedule(0)),
              metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall()])
model.summary()

[2022-06-13 00:04:47,207] INFO: Learning rate: 0.001000


Model: "ParticleNet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 mask (InputLayer)              [(None, 50, 1)]      0           []                               
                                                                                                  
 tf.math.not_equal_4 (TFOpLambd  (None, 50, 1)       0           ['mask[0][0]']                   
 a)                                                                                               
                                                                                                  
 tf.cast_8 (TFOpLambda)         (None, 50, 1)        0           ['tf.math.not_equal_4[0][0]']    
                                                                                                  
 tf.math.equal_4 (TFOpLambda)   (None, 50, 1)        0           ['tf.cast_8[0][0]']    

In [ ]:
# Prepare model model saving directory.
import os
save_dir = 'model_checkpoints'
model_name = '%s_model.{epoch:03d}.h5' % model_type
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = keras.callbacks.ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = keras.callbacks.LearningRateScheduler(lr_schedule)
progress_bar = keras.callbacks.ProgbarLogger()
callbacks = [checkpoint, lr_scheduler, progress_bar]

In [ ]:
#val_sample_weights = val_targets*class_weight[1] + (1-val_targets)*class_weight[0]
#test_sample_weights = val_targets*class_weight[1] + (1-val_targets)*class_weight[0]

In [ ]:
model.fit(
    [train_data["points"], train_data["features"], train_data["mask"]], 
    train_label,
    batch_size=batch_size,
    epochs=10, # --- train only for 1 epoch here for demonstration ---
    validation_data=([val_data["points"], val_data["features"], val_data["mask"]], val_label),
    shuffle=True,
    #callbacks=callbacks
)

Epoch 1/10
271/271 [==============================] - 65s 231ms/step - loss: 0.3496 - accuracy: 0.8622 - precision_2: 0.8691 - recall_2: 0.5769 - val_loss: 0.4705 - val_accuracy: 0.7701 - val_precision_2: 0.5516 - val_recall_2: 0.8193
Epoch 2/10
271/271 [==============================] - 61s 224ms/step - loss: 0.3158 - accuracy: 0.8794 - precision_2: 0.8886 - recall_2: 0.6331 - val_loss: 0.3362 - val_accuracy: 0.8729 - val_precision_2: 0.9240 - val_recall_2: 0.5794
Epoch 3/10
271/271 [==============================] - 60s 223ms/step - loss: 0.3128 - accuracy: 0.8809 - precision_2: 0.8932 - recall_2: 0.6354 - val_loss: 0.3226 - val_accuracy: 0.8771 - val_precision_2: 0.8398 - val_recall_2: 0.6766
Epoch 4/10
271/271 [==============================] - 61s 224ms/step - loss: 0.3119 - accuracy: 0.8810 - precision_2: 0.8937 - recall_2: 0.6354 - val_loss: 0.3116 - val_accuracy: 0.8814 - val_precision_2: 0.9131 - val_recall_2: 0.6227
Epoch 5/10
271/271 [==============================] - 60s 22

KeyboardInterrupt: ignored